# Association Rule Learning

Association Rule Learning helps us to identify the correlation between a set of items. For example, in a grocery, we can identify two items that people usually buy together. Based on a real stroy, it was identified that baby diapers and beer bottles are bought quite frequently together. Thus, this association information can be leveraged and the products can be placed widely apart so that the person has to walk through a lot of other products and thus, can also buy something that was not planned.

There are two methods to identify this associative relationship.
1. Apriori
2. Eclat

### Import libraries and data

In [1]:
import pandas as pd
import numpy as np
from apyori import apriori

In [2]:
dataset = pd.read_csv('Market_Basket_Optimisation.csv')
dataset.head(5)

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Apriori

**Key terms**
1. **Support (X)** = Number of documents with X / Total number of documents
2. **Confidence (X -> Y)** = Number of documents with Y and X / Number of documents with X
3. **Lift (X -> Y)** = Confidence (X -> Y) / Support (X)

The algorithm is based on `prior` knowledge. 

**Algorithm**
1. Decide minimum values for confidence and support
2. Consider all transactions with support more than minimum support
3. Take all rules that have confidence more than minimum confidence
4. Sort the rules by decreasing life

I'll first define certain constants to be used for the algorithm.

In [3]:
MIN_SUPPORT = 0.003
MIN_CONFIDENCE = 0.2
MIN_LIFT = 3
TOTAL_LISTS = dataset.shape[0]

Next, I'll extract all the unqiue items that are in the complete dataset.

In [4]:
items = set()
item_docs = dict()
for i in range(dataset.shape[0]):
    for j in range(dataset.shape[1]):
        item = dataset.iloc[i,j]
        if type(item) == str:
            items.add(item)
            if item in item_docs:
                item_docs[item].append(i)
            else:
                item_docs[item] = [i]

Given that I have the list of all items in all lists, the next step is to calculate the `support` for each item. For this, I create a dictionary where the key would be the item and the value for that key would be the support for that item.

In [5]:
support = {}
for item in items:
    support[item] = len(item_docs[item]) / TOTAL_LISTS

Next, I iterate over all the possible pair of items and extract the confidence and lift. After checking that the minimum values are met for support, confidence and list, I append the pair to the result list.
Finally, I sort the result list based on the decreasing order of the lift.

In [6]:
result = []
index = 0
for item1 in items:
    for item2 in items:
        if item1 != item2:
            common_count = len(np.intersect1d(item_docs[item1], item_docs[item2]))
            confidence = common_count / len(item_docs[item1])
            lift = confidence / support[item2]
            if (confidence >= MIN_CONFIDENCE and support[item1] >= MIN_SUPPORT and lift >= MIN_LIFT):
                result.append({})
                result[index]['firstItem'] = item1
                result[index]['secondItem'] = item2
                result[index]['lift'] = lift
                index += 1
associations = sorted(result, key = lambda x: x['lift'], reverse = True)

I can now just print the top 10 associations that we derived using the **Apriori algorithm**.

In [7]:
for i in range(10):
    print('If someone buys \033[1m{}\033[0m, they are also likely to buy \033[1m{}\033[0m with a lift of \033[1m{}\033[0m'
          .format(associations[i]['firstItem'], associations[i]['secondItem'], associations[i]['lift']))

If someone buys hand protein bar, they are also likely to buy protein bar with a lift of 12.45157719977864
If someone buys chocolate bread, they are also likely to buy red wine with a lift of 8.886255924170616
If someone buys pet food, they are also likely to buy red wine with a lift of 8.70490376245285
If someone buys sparkling water, they are also likely to buy hot dogs with a lift of 7.880220646178093
If someone buys dessert wine, they are also likely to buy salmon with a lift of 5.717552887364208
If someone buys fromage blanc, they are also likely to buy honey with a lift of 5.178127589063794
If someone buys light cream, they are also likely to buy chicken with a lift of 4.843304843304844
If someone buys pasta, they are also likely to buy escalope with a lift of 4.700185158809286
If someone buys burger sauce, they are also likely to buy chicken with a lift of 4.545454545454545
If someone buys pasta, they are also likely to buy shrimp with a lift of 4.51449390147315


### Eclat

**Key terms**
1. **Support (X and Y)** = Number of documents with X and Y both / Total number of documents

**Algorithm**
1. Decide minimum values for support
2. Consider all transactions with support more than minimum support
4. Sort the rules by decreasing support

I calculate the support for each pair of values and then sort them in the decreasing order based on `support`.

In [8]:
support = []
index = 0
for item1 in items:
    for item2 in items:
        if item1 != item2:
            common_count = len(np.intersect1d(item_docs[item1], item_docs[item2]))
            curr_support = common_count / TOTAL_LISTS
            if (curr_support >= MIN_SUPPORT):
                support.append({})
                support[index]['firstItem'] = item1
                support[index]['secondItem'] = item2
                support[index]['support'] = curr_support
                index += 1
associations = sorted(support, key = lambda x: x['support'], reverse = True)

I can now just print the top 10 associations that we derived using the **Eclat algorithm**.

In [9]:
for i in range(10):
    print('If someone buys \033[1m{}\033[0m, they are also likely to buy \033[1m{}\033[0m with a support of \033[1m{}\033[0m'
          .format(associations[i]['firstItem'], associations[i]['secondItem'], associations[i]['support']))

If someone buys spaghetti, they are also likely to buy mineral water with a support of 0.05973333333333333
If someone buys mineral water, they are also likely to buy spaghetti with a support of 0.05973333333333333
If someone buys chocolate, they are also likely to buy mineral water with a support of 0.05266666666666667
If someone buys mineral water, they are also likely to buy chocolate with a support of 0.05266666666666667
If someone buys eggs, they are also likely to buy mineral water with a support of 0.05093333333333333
If someone buys mineral water, they are also likely to buy eggs with a support of 0.05093333333333333
If someone buys milk, they are also likely to buy mineral water with a support of 0.048
If someone buys mineral water, they are also likely to buy milk with a support of 0.048
If someone buys ground beef, they are also likely to buy mineral water with a support of 0.040933333333333335
If someone buys mineral water, they are also likely to buy ground beef with a supp